In [1]:
from operator import add, sub
from time import sleep
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.sql.types import StructType

import pyspark.sql.types as tp
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt
%matplotlib inline

In [42]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
ssc = StreamingContext(sc, 10)

## basic example with list values

In [43]:
from pyspark.context import SparkContext
from pyspark.streaming import StreamingContext
from time import sleep

#ssc = StreamingContext(sc, 2)
 

In [44]:
rddQueue = []
for i in range(4):
    rddQueue += [ssc.sparkContext.parallelize([j for j in range(1, 21)],10)]


In [45]:
inputStream = ssc.queueStream(rddQueue)
mappedStream = inputStream.map(lambda x: (x % 10, 1))
reducedStream = mappedStream.reduceByKey(lambda a, b: a + b)
reducedStream.pprint()

In [46]:
ssc.start()
#sleep(6)
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2022-06-27 14:12:40
-------------------------------------------
(8, 2)
(0, 2)
(1, 2)
(9, 2)
(2, 2)
(3, 2)
(4, 2)
(5, 2)
(6, 2)
(7, 2)

-------------------------------------------
Time: 2022-06-27 14:12:50
-------------------------------------------
(8, 2)
(0, 2)
(1, 2)
(9, 2)
(2, 2)
(3, 2)
(4, 2)
(5, 2)
(6, 2)
(7, 2)



## mapper example in Dstream

In [113]:
from pyspark.context import SparkContext
from pyspark.streaming import StreamingContext
from time import sleep


In [114]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
ssc = StreamingContext(sc, 10)

In [115]:
rddQueue = []
for i in range(4):
    rddQueue += [ssc.sparkContext.parallelize([j for j in range(1, 5)],10)]


In [116]:
inputStream = ssc.queueStream(rddQueue)
outputStream=inputStream.map(lambda x: x*2 )
#outputStream=outputStream.reduce(lambda x,y: x+y)
print(type(inputStream),type(outputStream))
outputStream.pprint()

<class 'pyspark.streaming.dstream.DStream'> <class 'pyspark.streaming.dstream.TransformedDStream'>


In [117]:
ssc.start()
#sleep(6)
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2022-06-27 14:27:50
-------------------------------------------
2
4
6
8

-------------------------------------------
Time: 2022-06-27 14:28:00
-------------------------------------------
2
4
6
8



## basic example sum in a RDD

In [107]:
from pyspark.context import SparkContext
from pyspark.streaming import StreamingContext
from time import sleep


In [108]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
ssc = StreamingContext(sc, 10)

In [109]:
rddQueue = []
for i in range(4):
    rddQueue += [ssc.sparkContext.parallelize([j for j in range(1, 5)],10)]


In [110]:
inputStream = ssc.queueStream(rddQueue)
outputStream=inputStream.map(lambda x: x*2 )
outputStream=outputStream.reduce(lambda x,y: x+y)
print(type(inputStream),type(outputStream))
outputStream.pprint()

<class 'pyspark.streaming.dstream.DStream'> <class 'pyspark.streaming.dstream.TransformedDStream'>


In [111]:
ssc.start()
#sleep(6)
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2022-06-27 14:27:30
-------------------------------------------
20

-------------------------------------------
Time: 2022-06-27 14:27:40
-------------------------------------------
20



## basic local file read

In [223]:
sc=SparkContext("local[*]","StreamingExample")
ssc=StreamingContext(sc,10)
lines=ssc.textFileStream('data')

In [218]:
type(lines)

pyspark.streaming.dstream.DStream

In [219]:
lines.pprint()

In [220]:
# words=lines.flatMap(lambda x:x.split(" "))
# mapped_words=words.map(lambda x:(x,1))
# reduced_words=mapped_words.reduceByKey(lambda x,y:x+y)
# sorted_words=reduced_words.map(lambda x:(x[1],x[0])).transform(lambda x:x.sortByKey(False))
# sorted_words.pprint()


In [221]:
# only executes when new file is being added in the directory

ssc.start()
#ssc.awaitTermination()
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2022-06-27 15:00:10
-------------------------------------------

-------------------------------------------
Time: 2022-06-27 15:00:20
-------------------------------------------



In [222]:
#ssc.stop(stopSparkContext=True, stopGraceFully=True)

## basic local file read

In [261]:
sc=SparkContext("local[*]","StreamingExample")
ssc=StreamingContext(sc,10)
lines=ssc.textFileStream('data')

In [262]:
df=pd.read_csv('/home/scypyuser/2.streaming_fss/data/data1.txt')

In [263]:
array_stream = np.array_split(df,10)
#print(np.array(array_stream).shape)

In [264]:
# Input data
rddQueue = []

for i in array_stream:
    rddQueue += [ssc.sparkContext.parallelize([i])]
inputStream = ssc.queueStream(rddQueue)


In [265]:
inputStream.pprint()

In [266]:
ssc.start()
sleep(60)
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2022-06-27 15:11:50
-------------------------------------------
     word count from Wikipedia the free encyclopedia
0  the word count is the number of words in a doc...
1  is required to stay within certain numbers of ...
2  proceedings journalism and advertising Word co...
3  the translation job Word counts may also be us...
4  and reading speeds usually in words per minute...

-------------------------------------------
Time: 2022-06-27 15:12:00
-------------------------------------------
     word count from Wikipedia the free encyclopedia
5  six characters to a word is generally used Con...
6  In non fiction See also References Sources Ext...
7  This section does not cite any references or s...
8  reliable sources Unsourced material may be cha...
9  Variations in the operational definitions of h...

-------------------------------------------
Time: 2022-06-27 15:12:10
-------------------------------------------
      word count fro

## wordcount streaming

In [540]:
sc=SparkContext("local[*]","StreamingExample")
ssc=StreamingContext(sc,10)
#lines=ssc.textFileStream('data')

In [541]:
df=pd.read_csv('/home/scypyuser/2.streaming_fss/data/data1.txt',header=None)

In [542]:
arr=df.to_numpy()

In [543]:
array_stream = np.array_split(arr,10)


In [544]:
# Input data
rddQueue = []

for i in array_stream:
    #print(i)
    lx=[]
    for j in i:
        lx.append(j[0])
    print(lx)
    rddQueue += [ssc.sparkContext.parallelize(lx)]
inputStream = ssc.queueStream(rddQueue)


['word count from Wikipedia the free encyclopedia', 'the word count is the number of words in a document or passage of text Word counting may be needed when a text', 'is required to stay within certain numbers of words This may particularly be the case in academia legal', 'proceedings journalism and advertising Word count is commonly used by translators to determine the price for', 'the translation job Word counts may also be used to calculate measures of readability and to measure typing']
['and reading speeds usually in words per minute When converting character counts to words a measure of five or', 'six characters to a word is generally used Contents Details and variations of definition Software In fiction', 'In non fiction See also References Sources External links Details and variations of definition', 'This section does not cite any references or sources Please help improve this section by adding citations to', 'reliable sources Unsourced material may be challenged and removed']

In [545]:
lines1=inputStream.flatMap(lambda x: x.split(' '))
lines2=lines1.map(lambda x: (x,1))
#outputStream1=lines.map(lambda x: (x,1))
outputStream=lines2.reduceByKey(lambda x,y: x+y)
lines1.pprint()
lines2.pprint()
outputStream.pprint()

In [546]:
# lines=inputStream.flatMap(lambda x: x.split(" "))
# outputStream1=lines.map(lambda x: (x,1))
# #outputStream=outputStream1.reduceByKey(lambda x,y: x+y)
# outputStream1.pprint()

In [547]:
#inputStream.window(20,10).foreachRDD(get_count)

ssc.start()
sleep(2)
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2022-06-27 16:09:00
-------------------------------------------
word
count
from
Wikipedia
the
free
encyclopedia
the
word
count
...

-------------------------------------------
Time: 2022-06-27 16:09:00
-------------------------------------------
('word', 1)
('count', 1)
('from', 1)
('Wikipedia', 1)
('the', 1)
('free', 1)
('encyclopedia', 1)
('the', 1)
('word', 1)
('count', 1)
...

-------------------------------------------
Time: 2022-06-27 16:09:00
-------------------------------------------
('free', 1)
('of', 4)
('when', 1)
('numbers', 1)
('particularly', 1)
('counts', 1)
('a', 2)
('passage', 1)
('text', 2)
('Word', 3)
...

-------------------------------------------
Time: 2022-06-27 16:09:10
-------------------------------------------
and
reading
speeds
usually
in
words
per
minute
When
converting
...

-------------------------------------------
Time: 2022-06-27 16:09:10
-------------------------------------------
('and', 1)
('reading

## wordcount in each window

In [279]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext
ssc = StreamingContext(sc, 10)

In [280]:
lines=ssc.textFileStream('/home/scypyuser/2.streaming_fss/OVA_Omentum.csv')

In [281]:
#lines.for()

In [282]:
words=lines.flatMap(lambda x:type(x))
mapped_words=words.map(lambda x:(x,1))
mapped_words.pprint()
#reduced_words=mapped_words.reduceByKey(lambda x,y:x+y)


# sorted_words=reduced_words.map(lambda x:(x[1],x[0])).transform(lambda x:x.sortByKey(False))
# sorted_words.pprint()

In [283]:
ssc.start()
sleep(2)
ssc.stop(stopSparkContext=True, stopGraceFully=True)


-------------------------------------------
Time: 2022-06-27 15:20:20
-------------------------------------------

-------------------------------------------
Time: 2022-06-27 15:20:30
-------------------------------------------



## getting the shape of file

In [320]:
sc=SparkContext("local[*]","StreamingExample")
ssc=StreamingContext(sc,10)
#lines=ssc.textFileStream('data')

In [321]:
df=pd.read_csv('/home/scypyuser/2.streaming_fss/data/data1.txt')

In [322]:
array_stream = np.array_split(df,10)


In [323]:
# Input data
rddQueue = []

for i in array_stream:
    rddQueue += [ssc.sparkContext.parallelize([i])]
inputStream = ssc.queueStream(rddQueue)


In [324]:
lines1=inputStream.map(lambda x: str(x.shape))
#outputStream1=lines.map(lambda x: (x,1))
#outputStream=outputStream1.reduceByKey(lambda x,y: x+y)
lines1.pprint()

In [325]:
# lines=inputStream.flatMap(lambda x: x.split(" "))
# outputStream1=lines.map(lambda x: (x,1))
# #outputStream=outputStream1.reduceByKey(lambda x,y: x+y)
# outputStream1.pprint()

In [ ]:
#inputStream.window(20,10).foreachRDD(get_count)

ssc.start()
sleep(2)
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2022-06-27 15:30:30
-------------------------------------------
(5, 1)



In [8]:
#ssc = StreamingContext(sc, 10)

In [9]:
df=pd.read_csv('/home/scypyuser/OVA_Omentum.csv')

In [10]:
array_stream = np.array_split(df,10)
print(array_stream)

[     ID_REF  1007_s_at   121_at  1405_i_at  1438_at  1494_f_at  1552256_a_at  \
0    179922     5319.5    669.4      557.3    766.5      222.0         664.0   
1    117729     3193.1   4619.8      450.3    230.4      158.6         881.9   
2     53103     2615.5   3877.6      712.3    315.5      150.9         817.1   
3     89015     3339.7    557.9      674.0    456.8      343.6        1129.1   
4    117759     2025.7    747.3      805.6    352.5      243.1        1341.4   
5    152597     1644.8   3630.8      229.0    387.4      254.4         547.7   
6    152573     2314.9    954.7      127.0    925.8      221.7         944.5   
7    117584     4670.4    821.4     2280.4    241.3      248.1         576.3   
8     53099     3860.6    694.3       98.5   1270.9      232.5        2886.9   
9    102526      777.7    669.1      101.8    224.0      212.3        1387.0   
10   152650     1667.4    630.0      622.7     13.1      244.7        1004.7   
11    53118     2072.0    607.8       1

In [6]:
# df=pd.read_csv('/home/scypyuser/epsilon_train.csv')

In [8]:
# array_stream = np.array_split(df,10)
# print(array_stream)

In [27]:
# Input data
rddQueue = []

for i in array_stream:
    rddQueue += [ssc.sparkContext.parallelize([i])]
inputStream = ssc.queueStream(rddQueue)


AttributeError: 'NoneType' object has no attribute 'sc'

In [12]:
def get_count(x):
    xx=pd.DataFrame(x)
    print(xx.shape)
    return xx.shape

In [14]:
inputStream.window(20,10).foreachRDD(get_count)

ssc.start()
sleep(60)
ssc.stop(stopSparkContext=True, stopGraceFully=True)